# MNIST Digit Recognition
## Introduction
This function of this Jupyter notebook is to explain how the python script [Source link to script](https://github.com/kbarry91/Emerging-Technologies/tree/master/4-MNIST%20Digit%20Recognition%20Script) works and to discuss its performance.
<br>

The aim of the script as discribed in the project brief was to develope a __Digit recognition script__: a _Python script that takes an image ﬁle containing a handwritten digit and identiﬁes the digit using a supervised learning algorithm and the MNIST dataset._


## Folder Structure 
On inital download of the project you will notice the directory only has one folder 'testImages' and a 'digitrec.py' script
<br>
__testImages:__ Contain images of different sizes and types to test the algorithm.<br>
__digitrec.py:__ A python script containing the source code
![initial](https://i.imgur.com/8nkerH3.png)
Once the program has been compiled and run another folder 'models' will be generated. 
![after](https://i.imgur.com/LTGgkJI.png)
__models:__ Contains saved neural network model to increase performace and decrease wait time 

## Navigating The program
The program has been build to provide simplicity to the user. That is it hides all the complicated  